In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.subplots as sp
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import haversine_distances
from math import radians
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans,DBSCAN
from sklearn.metrics import silhouette_score


In [ ]:
#We'll be working on the month of April only
april=pd.read_csv("/content/sample_data/uber-raw-data-apr14.csv")
april.head()

,Date/Time,Lat,Lon,Base
0,4/1/2014 0:11:00,40.7690,-73.9549,B02512
1,4/1/2014 0:17:00,40.7267,-74.0345,B02512
2,4/1/2014 0:21:00,40.7316,-73.9873,B02512
3,4/1/2014 0:28:00,40.7588,-73.9776,B02512
4,4/1/2014 0:33:00,40.7594,-73.9722,B02512


# New Section

In [ ]:
april['Date'] =  pd.to_datetime(april['Date/Time'])
april['Hour'] = april.Date.dt.hour
april['Day_of_week'] = april.Date.dt.day_name()
mask_data=["Lat","Lon","Hour","Day_of_week","Type"]
april['Type'] = april['Day_of_week'].apply(lambda x: 'Friday' if x == 'Friday' else 'Saturday' if x == 'Saturday' else 'Sunday' if x == 'Sunday' else 'Weekday')
april_cleaned=april.loc[:,mask_data]

april_cleaned.head()


,Lat,Lon,Hour,Day_of_week,Type
0,40.7690,-73.9549,0,Tuesday,Weekday
1,40.7267,-74.0345,0,Tuesday,Weekday
2,40.7316,-73.9873,0,Tuesday,Weekday
3,40.7588,-73.9776,0,Tuesday,Weekday
4,40.7594,-73.9722,0,Tuesday,Weekday


In [ ]:
april_cleaned['Hour'].describe()

count    564516.000000
mean         14.465043
std           5.873925
min           0.000000
25%          10.000000
50%          16.000000
75%          19.000000
max          23.000000
Name: Hour, dtype: float64

In [ ]:
#Check if there is missing values
april_cleaned.isnull().sum()/april_cleaned.shape[0]

Lat            0.0
Lon            0.0
Hour           0.0
Day_of_week    0.0
Type           0.0
dtype: float64

In [ ]:
from pandas.api.types import CategoricalDtype
cats = [ 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
cat_type = CategoricalDtype(categories=cats, ordered=True)
april_cleaned['Day_of_week'] = april_cleaned['Day_of_week'].astype(cat_type)
april_cleaned = april_cleaned.sort_values(['Day_of_week', 'Hour'])
april_cleaned['Hour_2'] = april_cleaned['Hour'].apply(lambda x: 24 if x == 0 else x)
#Here I added a columns Hour_2 because the integer zero is not suitable for later usage.

april_cleaned.head()

,Lat,Lon,Hour,Day_of_week,Type,Hour_2
7785,40.7205,-73.9939,0,Monday,Weekday,24
7786,40.7407,-74.0077,0,Monday,Weekday,24
7787,40.7591,-73.9892,0,Monday,Weekday,24
7788,40.7419,-74.0034,0,Monday,Weekday,24
15857,40.7456,-73.9773,0,Monday,Weekday,24


In [ ]:

fig = px.histogram(april_cleaned,title="Daily Uber's orders", x="Day_of_week")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
april_cleaned.describe(include="all")

,Lat,Lon,Hour,Day_of_week,Type,Hour_2
count,564516.000000,564516.000000,564516.000000,564516,564516,564516.000000
unique,NaN,NaN,NaN,7,4,NaN
top,NaN,NaN,NaN,Wednesday,Weekday,NaN
freq,NaN,NaN,NaN,108631,345744,NaN
mean,40.740005,-73.976817,14.465043,NaN,NaN,14.971388
std,0.036083,0.050426,5.873925,NaN,NaN,5.634738
min,40.072900,-74.773300,0.000000,NaN,NaN,1.000000
25%,40.722500,-73.997700,10.000000,NaN,NaN,11.000000
50%,40.742500,-73.984800,16.000000,NaN,NaN,16.000000
75%,40.760700,-73.970000,19.000000,NaN,NaN,19.000000


In [ ]:
#Creating hour intervals
bins = [0,5,7,11,14,18,23]
label = ['Late night',"Early Morning", 'Morning', 'Lunch Break', 'Afternoon', 'Evening']
april_cleaned['Hour_interval']= pd.cut(april_cleaned['Hour'], bins=bins, labels=label,right=False)

hour_bin_counts = april_cleaned.groupby(['Type', 'Hour_interval']).size().unstack()


# Create pie charts for each 'Type'
row = 1
col = 1

for day_type in hour_bin_counts.index:
    data = hour_bin_counts.loc[day_type].reset_index()
    data.columns = ['Hour_interval', 'Count']

    fig = px.pie(data, names='Hour_interval', values='Count', title=f'Uber demand by time {day_type}')
    fig.show()


In [ ]:
hour_bin_counts.head()

Hour_interval,Late night,Early Morning,Morning,Lunch Break,Afternoon,Evening
Type,,,,,,
Friday,4308,4218,12922,9329,22490,31643
Saturday,8802,1548,6332,7560,18467,28790
Sunday,10805,1262,5989,7601,12977,11599
Weekday,11834,20946,58328,36312,96058,113747


In [ ]:
fig_map = px.scatter_mapbox(title="Uber demands in NYC", data_frame=april_cleaned,lat='Lat',lon='Lon',color='Hour',mapbox_style='open-street-map')
fig_map.show()
print("We already see the crazy high density of uber demands in the center of NY (Manhattan)")

Output hidden; open in https://colab.research.google.com to view.

# MACHINE LEARNING

## KMEANS

### Preprocessing

#### I will try first with one day then we will generalize for each of the week

In [ ]:
mask_friday=april_cleaned["Day_of_week"]=="Friday"
april_cleaned_friday=april_cleaned.loc[mask_friday,['Lat','Lon']].sample(10000)
april_cleaned_friday.head()

,Lat,Lon
257115,40.7346,-74.0100
167142,40.7079,-74.0060
257574,40.7432,-73.9934
231841,40.7334,-74.0084
53095,40.7429,-74.0077


k-means is not an ideal algorithm for latitude-longitude spatial data because it minimizes variance, not geodetic distance. We will use haversine distance

In [ ]:
X_friday=april_cleaned_friday[['Lat','Lon']].applymap(radians)
conversion_haversine=haversine_distances(X_friday)

In [ ]:
%%time
#Let's apply the elbow method

wcss =  []
k = []
for i in range (1,10):
    kmeans = KMeans(n_clusters= i, random_state = 0,n_init=2)
    kmeans.fit(X_friday)
    wcss.append(kmeans.inertia_)
    k.append(i)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))

WCSS for K=1 --> 0.009170994164292165
WCSS for K=2 --> 0.0060833080137734485
WCSS for K=3 --> 0.004419783804012485
WCSS for K=4 --> 0.003587325724056392
WCSS for K=5 --> 0.002860361206821607
WCSS for K=6 --> 0.0020071870118145317
WCSS for K=7 --> 0.0017446810414091176
WCSS for K=8 --> 0.0015713778836758344
WCSS for K=9 --> 0.0013704827756964497
CPU times: user 19.4 s, sys: 6.53 s, total: 26 s
Wall time: 3.53 s


In [ ]:
wcss_frame = pd.DataFrame(wcss)
k_frame = pd.Series(k)

# Create figure
fig_elbow= px.line(
    wcss_frame,
    x=k_frame,
    y=wcss_frame.iloc[:,-1]
)

# Create title and axis labels
fig_elbow.update_layout(
    yaxis_title="Inertia",
    xaxis_title="# Clusters",
    title="Inertia per cluster"
)

# Render
fig_elbow.show()

According to the elbow graph, the inertian tend to have less variation between 6 to 8 cluster. We will decide the best number of cluster after the silhouette method.

In [ ]:
#Silhouette method

# Computer mean silhouette score
sil = []
k = []

## Careful, you need to start at i=2 as silhouette score cannot accept less than 2 labels
for i in range (2,11):
    kmeans = KMeans(n_clusters= i, random_state = 0,n_init=2)
    kmeans.fit(X_friday)
    sil.append(silhouette_score(X_friday, kmeans.predict(X_friday)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.7356275837438028
Silhouette score for K=3 is 0.40904950570081433
Silhouette score for K=4 is 0.4102987279454935
Silhouette score for K=5 is 0.4571511163211144
Silhouette score for K=6 is 0.47446590833697466
Silhouette score for K=7 is 0.4255397521946529
Silhouette score for K=8 is 0.4370539314975989
Silhouette score for K=9 is 0.4421136456391003
Silhouette score for K=10 is 0.38463762924336853


In [ ]:
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

# Create figure
fig = px.bar(data_frame=cluster_scores,
             x=k,
             y=cluster_scores.iloc[:, -1]
            )

# Add title and axis labels
fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="# Clusters",
    title="Silhouette Score per cluster"
)

# Render
fig.show()


On the elbow method, the inertia had less variation between 6,7 or 8 cluster. According the Silhouette score, the cluster 6 is our best candidate.
Consequently, on th dataset narrowed fridays ("april_cleaned_friday"), the best number cluster in 7 for K means.
We will run the model with the correct K=7.



In [ ]:
kmeans = KMeans(n_clusters=7, random_state=0)
kmeans.fit(X_friday)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



KMeans(n_clusters=7, random_state=0)

In [ ]:
april_cleaned_friday["cluster_id"]=kmeans.labels_
april_cleaned_friday.head()

,Lat,Lon,cluster_id
257115,40.7346,-74.0100,1
167142,40.7079,-74.0060,1
257574,40.7432,-73.9934,1
231841,40.7334,-74.0084,1
53095,40.7429,-74.0077,1


In [ ]:
fig_map_friday = px.scatter_mapbox(title="Uber demands in NYC", data_frame=april_cleaned_friday,lat='Lat',lon='Lon',color='cluster_id',mapbox_style='open-street-map')
fig_map_friday.show()

Since this study has been done on Friday only, we will now iterate on each day of the week.

In [ ]:
df3=april_cleaned.loc[:,['Lat','Lon','Hour_2']]
df3.describe()

,Lat,Lon,Hour_2
count,564516.000000,564516.000000,564516.000000
mean,40.740005,-73.976817,14.971388
std,0.036083,0.050426,5.634738
min,40.072900,-74.773300,1.000000
25%,40.722500,-73.997700,11.000000
50%,40.742500,-73.984800,16.000000
75%,40.760700,-73.970000,19.000000
max,42.116600,-72.066600,24.000000


In [ ]:
type(df3)

pandas.core.frame.DataFrame

In [ ]:
df3 = df3.astype({'Hour_2': int})
print("After converting 'Hour_2' column into int:\n", df3.dtypes)

df3 = df3.sort_values(by=['Hour_2'], ascending=True)
df3.head(50)

After converting 'Hour_2' column into int:
 Lat       float64
Lon       float64
Hour_2      int64
dtype: object


,Lat,Lon,Hour_2
1020,40.7218,-73.9580,1
370965,40.7636,-73.9926,1
370964,40.7226,-73.9879,1
370963,40.7394,-74.0080,1
370962,40.6737,-73.9922,1
370961,40.7143,-73.9987,1
370960,40.7146,-74.0101,1
370959,40.7298,-73.9910,1
370958,40.7416,-73.9933,1
370957,40.7429,-74.0059,1


In [ ]:
#Create a dataframe for each of the week we preprocessed the data

day_of_week=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']


cluster_labels_df = pd.DataFrame()
sc = StandardScaler()

for day in day_of_week:
    mask_day=april_cleaned["Day_of_week"]==day
    dataframe_day=april_cleaned.loc[mask_day,['Lat','Lon','Hour_2']].sample(15000)
    dataframe_day['Day_of_week'] = day
    #print(dataframe_day.head())

    X=dataframe_day[['Lat','Lon']].applymap(radians)
    X_prepro=haversine_distances(X)
    # X_prepro = sc.fit_transform(X_prepro)


    wcss =  []
    k_elbow= []
    sil = []
    k_sil = []

    for i in range (1,10):
        kmeans = KMeans(n_clusters= i, random_state = 0,n_init=2)
        kmeans.fit(X_prepro)
        wcss.append(kmeans.inertia_)
        k_elbow.append(i)
        print("WCSS for K={} --> {}".format(i, wcss[-1]))

    wcss_frame = pd.DataFrame({'K': k_elbow, 'WCSS': wcss})

    # Create figure
    fig_elbow= px.line(
        wcss_frame,
        x=wcss_frame['K'],
        y=wcss_frame.iloc[:,-1]
    )

    # Create title and axis labels
    fig_elbow.update_layout(
        yaxis_title="Inertia",
        xaxis_title="# Clusters",
        title="Inertia per cluster"
    )

    # fig_elbow.show(renderer="notebook")

## Careful, you need to start at i=2 as silhouette score cannot accept less than 2 labels
    for i in range (2,11):
        kmeans = KMeans(n_clusters= i, random_state = 0,n_init=2)
        kmeans.fit(X_prepro)
        sil.append(silhouette_score(X_prepro, kmeans.predict(X_prepro)))
        k_sil.append(i)
        print("Silhouette score for K={} is {}".format(i, sil[-1]))

    cluster_scores=pd.DataFrame(sil)
    k_frame_sil = pd.Series(k_sil)

    # Create figure
    fig_silhouette = px.bar(data_frame=cluster_scores,
                x=k_frame_sil,
                y=cluster_scores.iloc[:, -1]
                )

    # Add title and axis labels
    fig_silhouette.update_layout(
        yaxis_title="Silhouette Score",
        xaxis_title="# Clusters",
        title="Silhouette Score per cluster"
    )

    # # Render
    # fig_silhouette.show(renderer="notebook")

    subplot = make_subplots(rows=1, cols=2, subplot_titles=("Elbow Plot {}".format(day), "Silhouette Score Plot {}".format(day),))
    subplot.add_trace(go.Scatter(fig_elbow.data[0]), row=1, col=1)
    subplot.update_xaxes(title_text="Clusters", row=1, col=1)
    subplot.update_yaxes(title_text="Inertia", row=1, col=1)
    subplot.add_trace(go.Bar(fig_silhouette.data[0]), row=1, col=2)
    subplot.update_yaxes(title_text="Silhouette Score", row=1, col=2)
    subplot.update_xaxes(title_text="Clusters", row=1, col=2)
    # Update layout
    subplot.update_layout(showlegend=False)

    # Show the subplot
    subplot.show()

    dataframe_day['Cluster_Label'] = kmeans.labels_
    cluster_labels_df = pd.concat([cluster_labels_df, dataframe_day])
    cluster_labels_df= cluster_labels_df.sort_values(by=['Hour_2'], ascending=True)

    fig_map_density_per_hour= px.density_mapbox(data_frame=cluster_labels_df, lat="Lat", lon="Lon", mapbox_style="open-street-map",
                        animation_frame = 'Hour_2',
                        zoom = 10, radius = 8, hover_name= cluster_labels_df['Cluster_Label'])

    fig_map_density_per_hour.show()

    fig_map_scatter_per_day = px.scatter_mapbox(title="Uber demands in NYC Kmeans {}".format(day), data_frame=cluster_labels_df,lat='Lat',lon='Lon',color=cluster_labels_df['Cluster_Label'],mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Hour_2")
    fig_map_scatter_per_day.show()


print("this is my final dataframe", cluster_labels_df.head())



Output hidden; open in https://colab.research.google.com to view.

According to the elbow and silhouette score above, we can determine the number of clusters for each day:
- Monday has 7 clusters
- Tuesday has 7 clusters
- Wednesday has 6 clusters
- Thursday has 8 clusters
- Friday has 6 clusters
- Saturday has 6 clusters
- Sunday has 7 clusters

In [ ]:
fig_scattermapbox_per_day = px.scatter_mapbox(title="Uber demands in NYC Kmeans per day", data_frame=cluster_labels_df,lat='Lat',lon='Lon',color=cluster_labels_df['Cluster_Label'],mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Day_of_week")
fig_scattermapbox_per_day.show()



Output hidden; open in https://colab.research.google.com to view.

# DB SCAN

### Baseline

In [ ]:
X = april_cleaned.copy().sample(10000)
X = X.iloc[:,:2]
X_db_processed = X[['Lat', 'Lon']].applymap(radians)
X_db_processed = haversine_distances(X_db_processed)

# DBSCAN
# epsilon: radius within which neighboring points are considered part of the same cluster (density).
# min_samples: minimum number of points required to form a dense region (core point)
# Ball-tree algorithm: used for non euclidian distances

db = DBSCAN(eps=0.005, min_samples=10, algorithm="ball_tree") # instantiate
db.fit(X_db_processed) # fit
X['Cluster_DBSCAN'] = db.labels_ # get predictions (clusters)

# Create the scatter mapbox plot
fig = px.scatter_mapbox(X, lat='Lat', lon='Lon', width = 800, color=X['Cluster_DBSCAN'].astype('str'), hover_data=['Cluster_DBSCAN'], mapbox_style='open-street-map')
fig.update_layout(title='DBSCAN Clustering (all days mixed test)')
# fig.show()

In [ ]:

#Create a dataframe for each of the week puis convertir

day_of_week=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

cluster_labels_df_dbscan = pd.DataFrame()

for day in day_of_week:
    mask_day=april_cleaned["Day_of_week"]==day
    dataframe_day_db=april_cleaned.loc[mask_day,['Lat','Lon']].sample(9000)
    dataframe_day_db['Day_of_week'] = day
    #print(dataframe_day.head())

    X=dataframe_day_db[['Lat','Lon']].applymap(radians)
    X_prepro_db=haversine_distances(X)

    db = DBSCAN(eps=0.005, min_samples=10, algorithm="ball_tree") # instantiate
    db.fit(X_prepro_db) # fit
    dataframe_day_db['Cluster_Label'] = db.labels_
    cluster_labels_df_dbscan = pd.concat([cluster_labels_df_dbscan, dataframe_day_db])

print("this is my final dataframe", cluster_labels_df_dbscan.head())

this is my final dataframe             Lat      Lon Day_of_week  Cluster_Label
382829  40.7290 -73.9830      Monday              0
66961   40.7448 -73.9797      Monday              0
141386  40.7518 -74.0038      Monday              0
266384  40.6459 -73.7768      Monday              1
103330  40.6992 -73.9940      Monday              2


In [ ]:
fig_map_dbscan = px.scatter_mapbox(title="Uber demands in NYC - DBScan", data_frame=cluster_labels_df_dbscan,lat='Lat',lon='Lon',color=cluster_labels_df_dbscan['Cluster_Label'],mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Day_of_week")
fig_map_dbscan.show()

In [ ]:
cluster_labels_df_dbscan.head()

,Lat,Lon,Day_of_week,Cluster_Label
382829,40.7290,-73.9830,Monday,0
66961,40.7448,-73.9797,Monday,0
141386,40.7518,-74.0038,Monday,0
266384,40.6459,-73.7768,Monday,1
103330,40.6992,-73.9940,Monday,2


In [ ]:
# Count number of outliers (cluster -1)
outliers = cluster_labels_df_dbscan[cluster_labels_df_dbscan['Cluster_Label'] == -1]
percentage_outliers = len(outliers) * 100 / len(cluster_labels_df_dbscan)
print(f"Percentage of outliers: {round(percentage_outliers)}%, we can remove them.")

# Remove them
final_df_dbscan = cluster_labels_df_dbscan.drop(outliers.index)

Percentage of outliers: 5%, we can remove them.


In [ ]:
fig_map_dbscan_final = px.scatter_mapbox(title="Uber demands in NYC wihtout outliers - DBScan", data_frame=final_df_dbscan,lat='Lat',lon='Lon',color=final_df_dbscan['Cluster_Label'],mapbox_style='open-street-map',color_continuous_scale='turbo',animation_frame = "Day_of_week")
fig_map_dbscan_final.show()

DBSCAN seems very good for finding hotspots. Removal of outliers makes clustering much better than with KMeans.
It eliminates the problem of not dense but very long zones that we had with KMeans.
We still have the central zone, which should be a bit smaller if we want drivers to be always less than 10min from a demand.

# CONCLUSION

The best method for our problem is DBSCAN, for the following reasons:
- unknown number of clusters for each day and hour
- many outliers far around New York city
- various densities depending on the zone in the city
- irregular shapes of clusters

However, the central zone around Manhattan (the densest one) should be clustered more, in order to respect the time asked (less than 10min driving). Or another solution would be to use KMEANS on the densest part of Manhattan which had very nice little clusters, but remove outliers before.
